In [ ]:
import requests
import os
import json
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
from auth_script import *

In [ ]:
base_url = "http://test311api.cityofchicago.org/open311/v2/"
services = base_url + 'services.json'
service_requests = base_url + 'requests.json'
s = requests.Session()

# building initial dataframe structure
service_requests_df = pd.DataFrame(columns=['service_request_id', 'status', 'service_name', 'service_code',
       'requested_datetime', 'updated_datetime', 'address', 'lat', 'long',
       'extended_attributes', 'notes'])

# Set up initial headers
page_num = 3501
service_params = {
    'page': page_num,
    'page_size': 200,
    'extensions': 'true'
}
service_requests_payload = s.get(service_requests, params=service_params).json()
results = []

# Get all data from REST API
while len(service_requests_payload) == 200:
    # store the current payload in a list
    results.extend(service_requests_payload)

    # update the page number
    page_num += 1
    service_params['page'] = page_num

    try:
        # generate a new payload
        service_requests_payload = s.get(service_requests, params=service_params).json()
    except:
        s = requests.Session()
        service_requests_payload = s.get(service_requests, params=service_params).json()

    if page_num % 10 == 0:
        print(f'Current Page: {page_num}')
    
    if page_num % 500 == 0:
        json.dump(results, open(f'./temp/service_requests_{page_num-500}-{page_num}.json', 'w'), ensure_ascii=False, indent=4)
        results = []
# Once finished with the loop, we'll need to write the last bit of data
json.dump(results, open(f'./temp/service_requests_{page_num}.json', 'w'), ensure_ascii=False, indent=4)

In [ ]:
def merge_json(temp_path, new_path, new_file_name):
    results = []
    for f in os.listdir(temp_path):
        with open(f'./temp/{f}') as infile:
            results.extend(json.load(infile))
    with open(f'./data/{new_file_name}', 'w+') as outfile:
        json.dump(results, outfile)

In [ ]:
merge_json('./temp/', './data/', 'service_requests_full.json')

In [ ]:
# get services list
services_list = requests.get(services)

In [ ]:
# create services df
services_list_df = pd.json_normalize(services_list.json())

In [ ]:
def expand_list(df, list_column, new_column): 
    lens_of_lists = df[list_column].apply(len)
    origin_rows = range(df.shape[0])
    destination_rows = np.repeat(origin_rows, lens_of_lists)
    non_list_cols = (
      [idx for idx, col in enumerate(df.columns)
       if col != list_column]
    )
    expanded_df = df.iloc[destination_rows, non_list_cols].copy()
    expanded_df[new_column] = (
      [item for items in df[list_column] for item in items]
      )
    expanded_df.reset_index(inplace=True, drop=True)
    return expanded_df

In [ ]:
%%time 
# load json data
sr_df = pd.json_normalize(json.load(open('./data/service_requests_full.json', 'r')))

# remove duplicates from errant data gathering
sr_df.drop_duplicates('service_request_id', inplace=True)

In [ ]:
# Filling the np.nans in the notes column because they were throwing an error
sr_df.loc[sr_df['notes'].isna(), 'notes'] = sr_df.loc[sr_df['notes'].isna(), 'notes'].apply(lambda x: [{'NA': 'NA'}])

In [ ]:
%%time
# expanding the notes section out to individual rows. This data frame now has 'per action' granularity
sr_notes_df = pd.json_normalize(expand_list(sr_df, 'notes', 'notes_indiv')['notes_indiv'])

# Ensuring that the work orders are attached to each row
sr_notes_df['extended_attributes.service_request_number'] = sr_notes_df['extended_attributes.service_request_number'].fillna(method='ffill')

In [ ]:
# Putting the two dfs together, though I can probably keep these separate and join them in Qlik
sr_notes_concat_df = pd.concat([expand_list(sr_df, 'notes', 'notes_indiv'), sr_notes_df], axis=1)

In [ ]:
services_list_df.to_csv('./data/services.csv', index=False)

In [ ]:
sr_df.to_csv('./data/service_requests.csv', index=False)

In [ ]:
sr_notes_df.to_csv('./data/service_requests_notes.csv', index=False)

In [ ]:
sr_notes_concat_df.to_csv('./data/service_requests_notes_concat.csv', index=False)